Defination and Loading MOdel


EMOTION_CLASSIFIER

In [1]:
#ALL HEADER FILES

import nltk
nltk.download('stopwords')
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict

import pickle as p

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize

from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag


from google.colab import drive
drive.mount('/content/gdrive')

datapath = '/content/gdrive/Shared drives/NLP/Project_Files/'
modelpath = '/content/gdrive/Shared drives/NLP/Project_Files/Models/EmotionClassifier'


# LOADING TOKENIZER
f = open(datapath + 'amit_normal_tokenizer_prompts.dat', 'rb')
tokenizer=p.load(f)
f.close()

# MODEL LOADED and EMOJI2VEC and WORD2VEC
import gensim.models as gsm
e2v = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/emoji2vec.bin', binary=True)
model = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/GoogleNews-vectors-negative300.bin', binary=True)

from keras.models import model_from_json
# load json and create model
json_file = open(datapath+"AMIT_epoch2W.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(datapath+"AMIT_epoch2W.h5")
print("Loaded model from disk")

import numpy as np
import scipy

 
emodict = {0:'amusement', 1:'satisfaction', 2:'optimism', 3:'pride in achievement', 4:'contentment',
           5:'anger', 6:'fear', 7:'disgust', 8:'sadness', 9:'contempt'}
emotion_label = {
		"fear" : ['😳', '😰', '😨', '😬'],
		"anger" : ['😱', '😡', '😤', '😒', '😑', '😠'],
		"disgust" :['😭', '😩', '🤦', '😯', '🤢', '😖'],
		"sadness" :['😫', '😭', '😞'] ,
		"contempt" :['😔', '💔', '😏'],
		"amusement" :['😲'],
    "pride in achievement" :['👌', '👏', '👍'],
    "satisfaction" :['😊', '🙏', '😔', '😎'],
    "optimism" :['💪', '😎', '🛐'],
    "contentment" :['😇', '😘', '😂', '😁', '💃'],
    "none": ['none']  
        
	  }


maxlen = 1000

def emoji_adding(sentence):
  sentence_e=[sentence]
  sequences_test = tokenizer.texts_to_sequences(sentence_e)
  X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)
  preds = loaded_model.predict_classes(X_test, verbose=0)
  emotion=emodict.get(preds[0])
  print(emotion)
  
  shorlisted_emojis=np.array(emotion_label[emotion])
  #print(shorlisted_emojis)
  emoji_vectors = [e2v[e] for e in shorlisted_emojis]
  #print(sentence)
  words = sentence.split(" ")
  vector_list = np.array([model[word] for word in words if word in model.vocab])
  avg_vector = np.average(vector_list,axis=0)
  distances = [scipy.spatial.distance.cosine(avg_vector,emoji_vector) for emoji_vector in emoji_vectors]
  selected_emoji=shorlisted_emojis[np.argmin(distances)]
  sen_with_emo=sentence+selected_emoji
  return sen_with_emo
  





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






Loaded model from disk


In [0]:
emotion_label = {
		"fear" : ['😳', '😰', '😨', '😬'],
		"anger" : ['😱', '😡', '😤', '😒', '😑', '😠'],
		"disgust" :['😭', '😩', '🤦', '😯', '🤢', '😖'],
		"sadness" :['😫', '😭', '😞'] ,
		"contempt" :['😔', '💔', '😏'],
		"amusement" :['😲'],
    "pride in achievement" :['👌', '👏', '👍'],
    "satisfaction" :['😊', '🙏', '😔', '😎'],
    "optimism" :['💪', '😎', '🛐'],
    "contentment" :['😇', '😘', '😂', '😁', '💃'],
    "none": ['none']  
        
	  }
all_emojis = []
for i in emotion_label:
    if i != 'none':
        all_emojis += emotion_label[i]
def direct_emoji_adding(sentence):
  
  shorlisted_emojis=np.array(all_emojis)
  #print(shorlisted_emojis)
  emoji_vectors = [e2v[e] for e in shorlisted_emojis]
  #print(sentence)
  words = sentence.split(" ")
  vector_list = np.array([model[word] for word in words if word in model.vocab])
  avg_vector = np.average(vector_list,axis=0)
  distances = [scipy.spatial.distance.cosine(avg_vector,emoji_vector) for emoji_vector in emoji_vectors]
  selected_emoji=shorlisted_emojis[np.argmin(distances)]
  sen_with_emo=sentence+selected_emoji
  return sen_with_emo
  


In [0]:
n=10
test = pd.read_csv(datapath + 'amit_test_emotion.csv')
test=test.dropna()
test=test[:n]
del test['Rating']
sentences=test['prompt']
actual_emotion=test['context']
for i in range(n):
    print("THIS IS RESULT FOR SENTENCE-",i+1)
    print("Actual EMOTIONS:", actual_emotion[i])
    emoji_prompt=emoji_adding(sentences[i])
    print(emoji_prompt)


THIS IS RESULT FOR SENTENCE- 1
Actual EMOTIONS: disgust
disgust
i felt guilty when i was driving home one night and a person tried to fly into my lane comma and didn t see me i honked and they swerved back into their lane comma slammed on their brakes comma and hit the water cones🤦
THIS IS RESULT FOR SENTENCE- 2
Actual EMOTIONS: contentment
pride in achievement
my mother stopped by my house one day and said she saw dogs on the road comma down from our house they were starving comma with ribs showing comma and it was a mother dog and her two small puppies of course comma my daughter wanted to bring them to our house comma so we could feed and help them we did comma and my heart went out to them comma as they were so sweet comma but really were in a bad shape👍
THIS IS RESULT FOR SENTENCE- 3
Actual EMOTIONS: contempt
contempt
i just broke up with my girlfriend comma we were together for years comma i feel so empty😔
THIS IS RESULT FOR SENTENCE- 4
Actual EMOTIONS: amusement
amusement
i rece

In [0]:
emojifier = np.vectorize(emoji_adding)

In [4]:
import os
os.listdir(new_data_path)

['Data_for_user_study - Sheet1.csv']

In [13]:
test = pd.read_csv(new_data_path + 'Data_for_user_study - Sheet1.csv')
test

,Context,Response
0,BERT based retrieval,NaN
1,NaN,NaN
2,Without emojis,NaN
3,I couldn ' t wait to go to the concert .,Oh that ' s cool ! What kind of concert is it ?
4,I ' m so sad because i ' ve read an article ab...,I ' m sorry to hear that . What do they get tr...
5,so my friend ' s baby fell down and scraped hi...,"Oh no , that ' s never good . Is he doing fine..."
6,so my friend ' s baby fell down and scraped hi...,The teachers were amazing with him . They taug...
7,my husband lost a job but i ' m hoping he can ...,Good of you . Times like these can be tough an...
8,my husband lost a job but i ' m hoping he can ...,Good of you . Times like these can be tough an...
9,i just moved to this neighborhood and some dum...,"Wow , thats scary . Did they have a gun ?"


In [22]:
n=10
new_data_path = '/content/gdrive/Shared drives/NLP/USER STUDY/'
test = pd.read_csv(new_data_path + 'Data_for_user_study - Sheet1.csv')
# test=test.dropna()
# test=test[:n]
# del test['Rating']
sentences=test['Response']
# actual_emotion=test['context']
test['Emojified'] = ''
for i in range(len(test)):
    try:
        test['Emojified'].iloc[i] = emoji_adding(test['Response'].iloc[i])
    except:
        pass
# emojifier(test['Response'])
# for i in range(n):
#     print("THIS IS RESULT FOR SENTENCE-",i+1)
#     print("Actual EMOTIONS:", actual_emotion[i])
#     emoji_prompt=emoji_adding(sentences[i])
#     print(emoji_prompt)

amusement
optimism
pride in achievement
pride in achievement
pride in achievement
pride in achievement
fear
pride in achievement
disgust
amusement
contempt
amusement
fear
pride in achievement
optimism
sadness
pride in achievement
disgust
pride in achievement
sadness
pride in achievement
pride in achievement
anger
contempt
optimism
sadness
optimism
optimism
contentment
satisfaction
optimism
pride in achievement
pride in achievement
fear
fear
optimism
sadness
optimism
fear
pride in achievement
amusement
fear
anger
sadness
disgust
satisfaction
amusement
amusement
pride in achievement
optimism


In [23]:
test['Emojified']

0                                                      
1                                                      
2                                                      
3      Oh that ' s cool ! What kind of concert is it ?😲
4     I ' m sorry to hear that . What do they get tr...
5     Oh no , that ' s never good . Is he doing fine...
6     The teachers were amazing with him . They taug...
7     Good of you . Times like these can be tough an...
8     Good of you . Times like these can be tough an...
9            Wow , thats scary . Did they have a gun ?😬
10                  how awesome . where do they live ?👍
11    Wow , that sucks . Sounds like it ' s your bos...
12    Oh thats a blessing . As long as the baby is a...
13    You sound lonely . Do you not have any friends ?😔
14    That actually sounds like a good idea . I coul...
15                 Is it just spiders that scare you ?😬
16    Oh wow ! That ' s great . Did she say how it w...
17    I ' ve had those experiences as well . My 

In [0]:
# test.to_csv(new_data_path + 'Emojified_for_use_study.csv', index=False)

In [0]:
new_data_path = '/content/gdrive/Shared drives/NLP/USER STUDY/'
test = pd.read_csv(new_data_path + 'Data_for_user_study - Sheet1.csv')
# test=test.dropna()
# test=test[:n]
# del test['Rating']
sentences=test['Response']
# actual_emotion=test['context']
test['EmojifiedDirect'] = ''
for i in range(len(test)):
    try:
        test['EmojifiedDirect'].iloc[i] = direct_emoji_adding(test['Response'].iloc[i])
    except:
        pass
# emojifier(test['Response'])
# for i in range(n):
#     print("THIS IS RESULT FOR SENTENCE-",i+1)
#     print("Actual EMOTIONS:", actual_emotion[i])
#     emoji_prompt=emoji_adding(sentences[i])
#     print(emoji_prompt)

In [31]:
test["EmojifiedDirect"]

0                                                      
1                                                      
2                                                      
3      Oh that ' s cool ! What kind of concert is it ?😔
4     I ' m sorry to hear that . What do they get tr...
5     Oh no , that ' s never good . Is he doing fine...
6     The teachers were amazing with him . They taug...
7     Good of you . Times like these can be tough an...
8     Good of you . Times like these can be tough an...
9            Wow , thats scary . Did they have a gun ?😂
10                  how awesome . where do they live ?😂
11    Wow , that sucks . Sounds like it ' s your bos...
12    Oh thats a blessing . As long as the baby is a...
13    You sound lonely . Do you not have any friends ?🙏
14    That actually sounds like a good idea . I coul...
15                 Is it just spiders that scare you ?👍
16    Oh wow ! That ' s great . Did she say how it w...
17    I ' ve had those experiences as well . My 

In [0]:
test.to_csv(new_data_path + 'EmojifiedDirect_for_use_study.csv', index=False)